Normalização do modelo BCS não linear

In [ ]:
import matplotlib.pyplot as plt
#import importlib
import numpy as np
#import subrotinas

#importlib.import_module('subrotinas')
%matplotlib tk


exec(compile(open('param.py', "rb").read(), 'param.py', 'exec')) #% Roda arquivo com parâmetros do modelo BCS
exec(compile(open('subrotinas.py', "rb").read(), 'subrotinas.py', 'exec')) #% Roda arquivo

### Variáveis de entrada
f $\in$ (35,65) Hz <br>
zc $\in$ (0,100)% <br>
pm $\in$ ($2\cdot 10^6$) <br>

### Variáveis de estado
pbh $\in$ ($10^5, 8.5 \cdot 10^6$) <br>
pwh $\in$ ($2 \cdot 10^6, 5.2\cdot10^6$) <br>
q $\in (10^{-2},3\cdot 10^{⁻2})$ <br>

### Saídas
pin $\in$ ($40, 60.5) \cdot 10^6$ <br>
H $\in$ ($350, 1000$) <br>

Função para retornar os valores de xc e x0 dado que<br>
$x_{min}<x<x_{max}$ <br>

Nova variável<br>

$xb=\frac{x-x0}{xc}$

Assim <br>

$x=xb\cdot xc+x0$

Fazendo com que a nova variável $0<xb<1$ <br>


In [ ]:
# Rotinas

Definindo os fatores de ajuste de escala a partir dos limites operacionais

In [ ]:
# Valores máximos e mínimos para normalização
#Entradas
f_lim=(30,75); zclim=(0,100);pmlim=(1e6,2e6);
#pbhlim=(100000,8.5e6);
pbhlim=(1e6,8.5e6);pwhlim=(2e6,5.2e6); qlim=(12/3600,55/3600)
H_lim=(350,1000);

#Hlim e Pin modificado
pinlim=(-3.28e12,-5.2e8)
H_lim=(-9.8e9,-110196);
#pinlim=(40e6,60.5e6)

In [ ]:
pm=2e6; #Simplificando pm fixo
unorm=np.array([Fnorm(f_lim),Fnorm(zclim)])
#xnorm= np.array([Fnorm(pbhlim),Fnorm(pwhlim),Fnorm(qlim), unorm[0,:],unorm[1,:]])
xnorm= np.array([Fnorm(pbhlim),Fnorm(pwhlim),Fnorm(qlim)])
ynorm=np.array([Fnorm(pinlim),Fnorm(H_lim)])
#pbh  - pressão na bomba
#pwh - Pressão de fundo do poço,
#q - vazão
#PI índice de produtividade do poço
#PinC  pressão na choke
# Pressão de manifold

Definindo as variáveis simbólicas

In [ ]:
# Criando simbolica
from casadi import *
nx = 3; nu = 2;
x = MX.sym("x",nx); # Estados
u = MX.sym("u",nu); # Exogena
dudt_max = MX.sym("dudt_max",2); # Exogena

In [ ]:
[pbh,pwh,q]=[x[0],x[1],x[2]]
x
#xnorm=np.array([[0,1],[0,1],[0,1]])

In [ ]:
pbh = (x[0]-xnorm[0,0])/xnorm[0,1]
pwh = (x[1]-xnorm[1,0])/xnorm[1,1]
q = (x[2]-xnorm[2,0])/xnorm[2,1]
# pwh = AplicaEscala(x[1],1);
# q = AplicaEscala(x[2],2)
fq = u[0]
zc = u[1]
#zc = AplicaEscala(x[4],4)

In [ ]:
# Entradas

fqref = u[0]*unorm[0,1]+unorm[0,0]; zcref = u[1]*unorm[1,1]+unorm[1,0]
pm=2e6
#fq=fqref; zc=zcref;
#pm=(pm-xnorm[0,0])/xnorm[0,1]
#pm = u[2];

In [ ]:
# Calculo do HEAD e delta de press�o
q0 = q/Cq*(f0/fq);
H0 = -1.2454e6*q0**2 + 7.4959e3*q0 + 9.5970e2;
H = CH*H0*(fq/f0)**2; # Head


Normalizando Pin (depende de x[0] e x[2])
Variação de H com u[0] e x[2] normalizados

In [ ]:
H=(H-ynorm[1,0])/ynorm[1,1]

In [ ]:
Pp = rho*g*H;
# Calculo da press�o de intake
F1 = 0.158*((rho*L1*q**2)/(D1*A1**2))*(mu/(rho*D1*q))**(1/4);
F2 = 0.158*((rho*L2*q**2)/(D2*A2**2))*(mu/(rho*D2*q))**(1/4);
P0 = -2.3599e9*q0**3 -1.8082e7*q0**2 +4.3346e6*q0 + 9.4355e4;
pin = pbh - rho*g*h1 - F1

funcPin=Function('funcPin',[x],[pin])

In [ ]:
ui=np.arange(0.1,1.01,0.1)
xi=np.arange(0.1,1.01,0.1)
zp=np.zeros((len(ui),len(xi)))
for i in range (0,len(ui)):
    for j in range(0,len(xi)):
        zp[i,j]=funcPin([ui[i],0,xi[j]])

#Limites de Pin
figPin, axP = plt.subplots(subplot_kw={"projection": "3d"})
norm = cm.colors.Normalize(vmin=0.1, vmax=1)
surfPin = axP.plot_surface(U, X, zp , cmap=cm.coolwarm,# norm=norm,
                       linewidth=0, antialiased=False)
# Customize the z axis.
#axP.set_zlim(0.19, 1)
axP.set_xlabel('Pbh')
axP.set_ylabel('q')
axP.set_zlabel('Pin')
#axP.zaxis.set_major_locator(LinearLocator(5))
# A StrMethodFormatter is used automatically
#axP.zaxis.set_major_formatter('{x:.02f}')
# Add a color bar which maps values to colors.
figPin.colorbar(surfPin, shrink=0.5, aspect=5)
plt.show()

In [ ]:
min_max_pin=[np.amin(zp), np.amax(zp)]
min_max_pin

In [ ]:
pinlim =min_max_pin
ynorm=np.array([Fnorm(pinlim),Fnorm(H_lim)])

pin = pbh - rho*g*h1 - F1
pin=(pin-ynorm[0,0])/ynorm[0,1]
funcPin=Function('funcPin',[x],[pin])

In [ ]:
zp=np.zeros((len(ui),len(xi)))
zp

In [ ]:
for i in range (0,len(ui)):
    for j in range(0,len(xi)):
        zp[i,j]=funcPin([ui[i],0,xi[j]])
zp

In [ ]:
#Limites de Pin
figPin, axP = plt.subplots(subplot_kw={"projection": "3d"})
norm = cm.colors.Normalize(vmin=0.1, vmax=1)
surfPin = axP.plot_surface(U, X, zp , cmap=cm.coolwarm,# norm=norm,
                       linewidth=0, antialiased=False)
# Customize the z axis.
#axP.set_zlim(0.19, 1)
axP.set_xlabel('Pbh')
axP.set_ylabel('q')
axP.set_zlabel('Pin')
#axP.zaxis.set_major_locator(LinearLocator(5))
# A StrMethodFormatter is used automatically
#axP.zaxis.set_major_formatter('{x:.02f}')
# Add a color bar which maps values to colors.
figPin.colorbar(surfPin, shrink=0.5, aspect=5)
plt.show()